## Model One

First attempt to build a ML model for the Kaggle Titanic competion. After exploring the data seems like the main features to consider are Sex and Pclass. After that the family related features SibSp and Parch also seem to have some effect onsurvival rates which is related to children having a a higher survial rate. Age only seems relevant in the bracket corresponding to children. Fare besides strongly correlated to Pclass, it also seems to correlate to families, in the sense that inside the same Pclass famileies are paying a higher fare per passanger.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing, impute, tree
from sklearn.experimental import enable_iterative_imputer

Since the number of features is fairly limited and most of them are categorical a decision tree seems to be a good model to use. For the cateforical features we will use a One Hot encoder. For SibSp and Parch we only consider to categories (i.e. family or no family). Fare and age are discretized. We disregard the rest of features.

In [4]:
#Load Train date

train = pd.read_csv('train.csv')

#Apply the encoders
x = np.array(train[['Sex']])
enc = preprocessing.OneHotEncoder(drop='if_binary', handle_unknown ='ignore')
p1 = enc.fit_transform(x).toarray()

x = np.array(train[['Pclass','Embarked']])
enc = preprocessing.OneHotEncoder(drop='if_binary', handle_unknown ='ignore', max_categories=3)
p2 = enc.fit_transform(x).toarray()

x = np.array(train[['SibSp','Parch']])
enc = preprocessing.OneHotEncoder(drop='if_binary', handle_unknown ='ignore', max_categories = 2)
p3 = enc.fit_transform(x).toarray()

x = np.array(train[['Fare']])
enc = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
p4 = enc.fit_transform(x)

# We asign -10 for missing age values to create its own category
imp = impute.SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = -10)
x = imp.fit_transform(np.array(train[['Age']]))
enc = preprocessing.KBinsDiscretizer(n_bins=9, encode='ordinal', strategy='uniform')
p5 = enc.fit_transform(x)

#Put all the encoded data together and train the model
x = np.hstack((p1,p2,p3,p4,p5))
y = np.array(train[['Survived']])
clf = tree.DecisionTreeClassifier(criterion='entropy').fit(x,y)

In [6]:
# Load the test data

test = pd.read_csv('test.csv')

# Encode the test data features as before

x = np.array(test[['Sex']])
enc = preprocessing.OneHotEncoder(drop='if_binary', handle_unknown ='ignore')
p1 = enc.fit_transform(x).toarray()

x = np.array(test[['Pclass','Embarked']])
enc = preprocessing.OneHotEncoder(drop='if_binary', handle_unknown ='ignore', max_categories=3)
p2 = enc.fit_transform(x).toarray()

x = np.array(test[['SibSp','Parch']])
enc = preprocessing.OneHotEncoder(drop='if_binary', handle_unknown ='ignore', max_categories = 2)
p3 = enc.fit_transform(x).toarray()

#Assign missing values of age with an IterativeImputer
imp = impute.IterativeImputer(missing_values=np.nan)
x = imp.fit_transform(np.array(test[['Fare']]))
enc = preprocessing.KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
p4 = enc.fit_transform(x)

imp = impute.SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = -10)
x = imp.fit_transform(np.array(test[['Age']]))
enc = preprocessing.KBinsDiscretizer(n_bins=9, encode='ordinal', strategy='uniform')
p5 = enc.fit_transform(x)

# Put the test data together and predict outcomes using the model we have trained
x = np.hstack((p1,p2,p3,p4,p5))
pred = clf.predict(x)

In [7]:
# We create file to submit  to Kaggle

test['Survived']= pred
test[['PassengerId','Survived']].to_csv('submission.csv',index=False)

This model achieved an accuracy of 0.74641.